In [1]:
# Import the dependencies
import pandas as pd
import gmaps
import requests
import numpy as np

# Import the API key
from config import g_key

In [2]:
# Store the CSV into a DataFrame
city_data_df = pd.read_csv("weather_database/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (Inches),Snow (Inches)
0,0,Praia,CV,14.92,-23.51,87.80,40,0,17.22,clear sky,0.00,0.0
1,1,Cape Town,ZA,-33.93,18.42,60.01,100,75,13.87,light rain,0.51,0.0
2,2,Busselton,AU,-33.65,115.33,62.01,73,25,18.66,scattered clouds,0.00,0.0
3,3,Rikitea,PF,-23.12,-134.97,76.53,85,75,10.04,light rain,0.42,0.0
4,4,Atuona,PF,-9.80,-139.03,80.47,71,6,11.12,clear sky,0.00,0.0


In [3]:
# Configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [4]:
# Review data types to ensure appropriateness
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (Inches)          float64
Snow (Inches)          float64
dtype: object

In [5]:
# Ensure completeness
city_data_df.count()

City_ID                566
City                   566
Country                561
Lat                    566
Lng                    566
Max Temp               566
Humidity               566
Cloudiness             566
Wind Speed             566
Current Description    566
Rain (Inches)          566
Snow (Inches)          566
dtype: int64

In [6]:
# Investigate Country with 5 missing values
city_data_df.loc[city_data_df["Country"].isnull()]

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (Inches),Snow (Inches)
98,98,Oranjemund,NaN,-28.55,16.43,58.66,62,19,14.00,few clouds,0.0,0.0
172,172,Walvis Bay,NaN,-22.96,14.51,61.00,83,0,10.71,clear sky,0.0,0.0
218,218,Henties Bay,NaN,-22.12,14.28,61.00,83,0,8.66,clear sky,0.0,0.0
362,362,Eenhana,NaN,-17.47,16.33,87.80,16,3,2.24,clear sky,0.0,0.0
406,406,Luderitz,NaN,-26.65,15.16,68.00,49,2,17.22,clear sky,0.0,0.0


In [7]:
# All NaN relate to the country Namibia (country code = NA), therefore replace all NaN with NA
city_data_df["Country"].fillna("NA", inplace=True)
city_data_df.count()

City_ID                566
City                   566
Country                566
Lat                    566
Lng                    566
Max Temp               566
Humidity               566
Cloudiness             566
Wind Speed             566
Current Description    566
Rain (Inches)          566
Snow (Inches)          566
dtype: int64

In [8]:
# Ask the customer to add a minimum and maximum temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))
want_rain = (input("Do you want it to be raining? (yes/no)"))
want_snow = (input("Do you want it to be snowing? (yes/no)"))

What is the minimum temperature you would like for your trip?45
What is the maximum temperature you would like for your trip?70
Do you want it to be raining? (yes/no)yes
Do you want it to be snowing? (yes/no)no


In [17]:
# Filter the dataset to find the cities that fit the criteria
if (want_rain == "yes") & (want_snow == "no"):
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                           (city_data_df["Max Temp"] >= min_temp) &
                                           (city_data_df["Rain (Inches)"] > 0)
                                          ]            
elif (want_rain == "no") & (want_snow == "yes"):
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                           (city_data_df["Max Temp"] >= min_temp) &
                                           (city_data_df["Snow (Inches)"] > 0)
                                          ]
elif (want_rain == "yes") & (want_snow == "yes"):
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                           (city_data_df["Max Temp"] >= min_temp) &
                                           (city_data_df["Rain (Inches)"] > 0) &
                                           (city_data_df["Snow (Inches)"] > 0) 
                                          ]
elif (want_rain == "no") & (want_snow == "no"):
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                           (city_data_df["Max Temp"] >= min_temp) &
                                           (city_data_df["Rain (Inches)"] == 0) &
                                           (city_data_df["Snow (Inches)"] == 0) 
                                          ]
    
else:
    print("issue")

In [18]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Cape Town,ZA,60.01,light rain,-33.93,18.42,
18,Sobolevo,RU,61.16,light rain,54.43,31.90,
36,Sfantu Gheorghe,RO,55.99,light rain,45.87,25.78,
45,Hermanus,ZA,57.00,light rain,-34.42,19.23,
85,Jamestown,US,69.01,moderate rain,42.10,-79.24,
97,Ketchikan,US,51.80,moderate rain,55.34,-131.65,
112,Bluff,NZ,47.32,light rain,-46.60,168.33,
118,Buy,RU,61.70,light rain,58.48,41.53,
134,Bedele,ET,60.04,light rain,8.46,36.35,
164,Dharchula,IN,63.52,light rain,29.85,80.53,


In [19]:
# Set parameters to search for a hotel
params = {
    "radius":5000,
    "type": "lodging",
    "key": g_key
    }

In [20]:
# Iterate through the DataFrame
for index, row in hotel_df.iterrows():
    
    # Get the lat and longs
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our lat and long
    base_url = base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()

    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Cape Town,ZA,60.01,light rain,-33.93,18.42,Southern Sun Waterfront Cape Town
18,Sobolevo,RU,61.16,light rain,54.43,31.90,
36,Sfantu Gheorghe,RO,55.99,light rain,45.87,25.78,Hotel Park Sfantu Gheorghe
45,Hermanus,ZA,57.00,light rain,-34.42,19.23,Misty Waves Boutique Hotel
85,Jamestown,US,69.01,moderate rain,42.10,-79.24,Hampton Inn & Suites Jamestown


In [22]:
# Create the pop up box framework
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and high of {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [28]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
# Create the output .csv file
output_data_file = "weather_database/WeatherPy_vacation.csv"

# Export WeatherPy_vacation into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")